In [77]:
# load relevant packages, set some display parameters
%reset -f

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RepeatedKFold
from math import sqrt
from scipy import stats
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, learning_curve
#import ifsg
import glob, os #I ADDED
from sklearn.svm import SVR, LinearSVR #I ADDED
from sklearn.tree import DecisionTreeRegressor #I ADDED
import pickle
import re
import random
import os
import warnings
import seaborn as sns
import gc
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

#ifsg.__version__

In [78]:
# # set paths
# x=os.path.splitext(os.path.split(os.path.splitext(os.path.split(os.getcwd())[0])[0])[0])
# path_dataset = str(x[0])+os.sep+"Data"+os.sep+"Datasets"+os.sep+"reports"

# print('path_dataset = ', path_dataset)

In [79]:
# load data
path_dataset = 'W:\\Work\\Assense\\ML\\chars_file'
reports_filename = os.path.join(path_dataset,'data_from_firestore_2403.csv')
reports_table = pd.read_csv(reports_filename)#,usecols=range(9))#, index_col=0,skiprows=[0,1,2])#,usecols=range(12))
reports_table.set_index('id',inplace=True)
reports_table.drop(columns=['Quantity vs Quality','Memory','Self-efficacy','Working memory',
                            'Quantitative Reasoning','Quantitative reasoning','Agreeableness'],inplace=True)
reports_table.rename(columns={'Attention: Coping with Distractions':'Coping with Distractions'},inplace=True)
reports_table.replace({-1:np.nan},inplace=True)
reports_table.count(axis=0)
#reports_table[~(reports_table['Purposefulness Vs. Hesitant']+reports_table['Decisiveness']).isnull()]
#observation_table.replace('Precision','Thoroughness',inplace=True)

company_name                           788
Time Management                        686
Teamwork                               380
Adaptability to changes                141
Decisiveness                           267
Purposefulness Vs. Hesitant            375
Adaptability to Changes                 39
Risk Taking                            168
Stress Tolerance                        95
Coping with Distractions               199
Goal orientation                       210
Coping with stress                      18
Methodicalness                         210
Performance IQ                          11
Thoroughness                           267
Empathy                                180
Processing Speed                       195
Performance under stress                55
Processing speed                        72
Perceptual Organization                418
Precision                              424
Monitoring                             424
Orderliness                            666
Performance

In [80]:
replace_dict = {'Adaptability to changes':'Adaptability to Changes',
                'Purposefulness Vs. Hesitant':'Decisiveness', 
                'Stress Tolerance':'Performance under Stress',
                'Coping with stress':'Performance under Stress',  
                'Performance under stress':'Performance under Stress',
                'Monitoring' :'Self Monitoring',
                'Precision':'Thoroughness',
                'Verbal reasoning':'Verbal Reasoning',
                'Performance IQ ':'Performance IQ',
                'Processing speed':'Processing speed',
                'Perceptual Organization':'Spatial reasoning'}
                #'Cognitive ability total score':'Intelligence Quotient'}

for rep in replace_dict.keys():
    reports_table[replace_dict[rep]][reports_table[replace_dict[rep]].isnull() & reports_table[rep].notnull()] = \
    reports_table[rep][reports_table[replace_dict[rep]].isnull() & reports_table[rep].notnull()]
    reports_table.drop(columns=[rep],inplace=True)

In [81]:
reports_table=reports_table[['company_name','Cognitive ability total score','Spatial reasoning','Verbal Reasoning','Processing Speed','Performance IQ',
                           'Time Management','Decisiveness','Thoroughness','Orderliness','Self Monitoring','Planning',
                           'Goal orientation','Methodicalness','Adaptability to Changes','Risk Taking','Performance under Stress',
                           'Empathy','Appropriate Behaviour Understanding','Conscientiousness','Emotional Understanding','Collegial Behavior','Teamwork']]
# 'Performance IQ', 'Coping with Distractions','Processing Speed',
# 'Empathy','Appropriate Behaviour Understanding','Conscientiousness','Emotional Understanding','Collegial Behavior'

reports_table.count(axis=0)

company_name                           788
Cognitive ability total score          270
Spatial reasoning                      684
Verbal Reasoning                       431
Processing Speed                       195
Performance IQ                         683
Time Management                        686
Decisiveness                           637
Thoroughness                           686
Orderliness                            666
Self Monitoring                        684
Planning                               686
Goal orientation                       210
Methodicalness                         210
Adaptability to Changes                180
Risk Taking                            168
Performance under Stress               281
Empathy                                180
Appropriate Behaviour Understanding    180
Conscientiousness                      180
Emotional Understanding                180
Collegial Behavior                     180
Teamwork                               380
dtype: int6

In [82]:
reports_table = reports_table.loc[reports_table['company_name'] == 'Mitsubishi']

In [83]:
Mit_db_r= reports_table[['Cognitive ability total score', 'Spatial reasoning', 'Verbal Reasoning', 'Processing Speed', 
                 'Performance IQ', 'Planning', 'Time Management', 'Self Monitoring',
                 'Goal orientation', 'Thoroughness','Orderliness', 'Methodicalness', 'Performance under Stress','Risk Taking', 
                 'Adaptability to Changes','Decisiveness']]

In [69]:
Mit_db_r.dropna(axis=0,subset=Mit_db_r.columns[1:],how='all')

,Cognitive ability total score,Spatial reasoning,Verbal Reasoning,Processing Speed,Performance IQ,Planning,Time Management,Self Monitoring,Goal orientation,Thoroughness,Orderliness,Methodicalness,Performance under Stress,Risk Taking,Adaptability to Changes,Decisiveness
id,,,,,,,,,,,,,,,,
5c1c54d7da0e492bb4580c7b,5.5,5.3,5.8,5.9,6.0,4.666667,4.000000,3.333333,5.333333,4.666667,3.000000,4.666667,3.000000,2.000000,4.000000,6.000000
5c1c55cfda0e492bb4580c7d,5.6,4.1,5.3,4.2,4.6,5.000000,4.500000,4.500000,4.000000,5.500000,3.000000,5.000000,1.500000,1.000000,5.000000,4.000000
5c2315fb58497715f43f3dd7,3.9,3.1,2.5,3.4,3.5,2.000000,3.500000,2.000000,1.500000,2.500000,3.000000,3.000000,1.000000,4.500000,2.000000,2.500000
5c2316ec58497715f43f3dd8,5.5,3.6,5.1,4.1,3.7,1.000000,3.500000,2.500000,1.500000,2.000000,2.500000,2.000000,1.000000,5.000000,2.500000,4.000000
5c380026ab15e621f457e791,5.0,4.2,4.3,4.6,4.8,3.666667,4.666667,3.000000,1.666667,4.000000,1.666667,2.000000,1.333333,3.666667,1.333333,2.000000
5c4a955bcd053b079445153a,5.1,5.2,4.8,4.8,5.5,4.000000,4.600000,4.200000,4.000000,5.400000,4.200000,5.250000,4.600000,1.000000,5.250000,4.000000
5c4aa60e9dd0fb08d0c4545c,4.0,3.8,3.9,3.0,3.9,3.800000,3.000000,3.000000,2.750000,2.750000,1.600000,2.750000,1.400000,5.500000,4.000000,3.000000
5c4aa6229dd0fb08d0c4545d,4.1,4.6,3.2,3.2,4.4,3.000000,4.000000,2.333333,3.000000,4.333333,2.000000,3.000000,1.666666,5.000000,1.333333,6.000000
5c4aa6349dd0fb08d0c4545e,3.8,3.5,3.8,3.8,3.9,2.200000,2.000000,2.300000,1.750000,2.900000,1.000000,1.500000,3.000000,4.500000,2.000000,3.800000


In [43]:
reports_table.head()

,company_name,Intelligence Quotient,Spatial reasoning,Verbal Reasoning,Processing Speed,Performance IQ,Time Management,Decisiveness,Thoroughness,Orderliness,Self Monitoring,Planning,Goal orientation,Methodicalness,Adaptability to Changes,Risk Taking,Performance under Stress,Empathy,Appropriate Behaviour Understanding,Conscientiousness,Emotional Understanding,Collegial Behavior,Teamwork
id,,,,,,,,,,,,,,,,,,,,,,,
5c1c54d7da0e492bb4580c7b,Mitsubishi,5.5,5.3,5.8,5.9,6.0,4.000000,6.0,4.666667,3.000000,3.333333,4.666667,5.333333,4.666667,4.000000,2.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN
5c1c55cfda0e492bb4580c7d,Mitsubishi,5.6,4.1,5.3,4.2,4.6,4.500000,4.0,5.500000,3.000000,4.500000,5.000000,4.000000,5.000000,5.000000,1.000000,1.500000,NaN,NaN,NaN,NaN,NaN,NaN
5c2315fb58497715f43f3dd7,Mitsubishi,3.9,3.1,2.5,3.4,3.5,3.500000,2.5,2.500000,3.000000,2.000000,2.000000,1.500000,3.000000,2.000000,4.500000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
5c2316ec58497715f43f3dd8,Mitsubishi,5.5,3.6,5.1,4.1,3.7,3.500000,4.0,2.000000,2.500000,2.500000,1.000000,1.500000,2.000000,2.500000,5.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
5c380026ab15e621f457e791,Mitsubishi,5.0,4.2,4.3,4.6,4.8,4.666667,2.0,4.000000,1.666667,3.000000,3.666667,1.666667,2.000000,1.333333,3.666667,1.333333,1.25,3.1875,2.5,4.5,5.25,NaN


In [9]:
reports_table_all = reports_table.dropna(axis=0).round(0)
reports_table_all.shape

(0, 23)

In [26]:
reports_table_all.head()

NameError: name 'reports_table_all' is not defined

In [84]:
reports_table_cog = Mit_db_r[['Cognitive ability total score','Spatial reasoning','Verbal Reasoning','Processing Speed','Performance IQ']]
reports_table_cog['Working Memory'] = 0.4*reports_table_cog['Cognitive ability total score']+0.6*reports_table_cog['Verbal Reasoning']
reports_table_cog.dropna(axis=0,how='any',inplace=True)
print(reports_table_cog.shape)
reports_table_cog
#.drop_duplicates()

(42, 6)


,Cognitive ability total score,Spatial reasoning,Verbal Reasoning,Processing Speed,Performance IQ,Working Memory
id,,,,,,
5c1c54d7da0e492bb4580c7b,5.5,5.3,5.8,5.9,6.0,5.68
5c1c55cfda0e492bb4580c7d,5.6,4.1,5.3,4.2,4.6,5.42
5c2315fb58497715f43f3dd7,3.9,3.1,2.5,3.4,3.5,3.06
5c2316ec58497715f43f3dd8,5.5,3.6,5.1,4.1,3.7,5.26
5c380026ab15e621f457e791,5.0,4.2,4.3,4.6,4.8,4.58
5c4a955bcd053b079445153a,5.1,5.2,4.8,4.8,5.5,4.92
5c4aa60e9dd0fb08d0c4545c,4.0,3.8,3.9,3.0,3.9,3.94
5c4aa6229dd0fb08d0c4545d,4.1,4.6,3.2,3.2,4.4,3.56
5c4aa6349dd0fb08d0c4545e,3.8,3.5,3.8,3.8,3.9,3.80


In [72]:
reports_table_cog = reports_table_cog.dropna(axis=0).round(0)
print('before:',reports_table_cog.shape)
print('after:',reports_table_cog.drop_duplicates().shape)

before: (42, 5)
after: (21, 5)


In [13]:
reports_table_cog.drop_duplicates().to_csv(path_dataset+os.sep+'unique_cog_reports.csv')

In [85]:
reports_table_wm = reports_table[['Planning', 'Time Management', 'Self Monitoring',
                 'Goal orientation', 'Thoroughness','Orderliness', 'Methodicalness', 'Performance under Stress','Risk Taking', 
                 'Adaptability to Changes','Decisiveness']]
reports_table_wm.dropna(axis=0,how='any',inplace=True)
print(reports_table_wm.shape)
reports_table_wm.head()

(38, 11)


,Planning,Time Management,Self Monitoring,Goal orientation,Thoroughness,Orderliness,Methodicalness,Performance under Stress,Risk Taking,Adaptability to Changes,Decisiveness
id,,,,,,,,,,,
5c1c54d7da0e492bb4580c7b,4.666667,4.000000,3.333333,5.333333,4.666667,3.000000,4.666667,3.000000,2.000000,4.000000,6.0
5c1c55cfda0e492bb4580c7d,5.000000,4.500000,4.500000,4.000000,5.500000,3.000000,5.000000,1.500000,1.000000,5.000000,4.0
5c2315fb58497715f43f3dd7,2.000000,3.500000,2.000000,1.500000,2.500000,3.000000,3.000000,1.000000,4.500000,2.000000,2.5
5c2316ec58497715f43f3dd8,1.000000,3.500000,2.500000,1.500000,2.000000,2.500000,2.000000,1.000000,5.000000,2.500000,4.0
5c380026ab15e621f457e791,3.666667,4.666667,3.000000,1.666667,4.000000,1.666667,2.000000,1.333333,3.666667,1.333333,2.0


In [74]:
reports_table_wm = reports_table_wm.dropna(axis=0).round(0)
print('before:',reports_table_wm.shape)
print('after:',reports_table_wm.drop_duplicates().shape)
#reports_table_wm.drop_duplicates()

before: (38, 11)
after: (36, 11)


In [86]:
tot_table = reports_table_cog.join(reports_table_wm)
tot_table

,Cognitive ability total score,Spatial reasoning,Verbal Reasoning,Processing Speed,Performance IQ,Working Memory,Planning,Time Management,Self Monitoring,Goal orientation,Thoroughness,Orderliness,Methodicalness,Performance under Stress,Risk Taking,Adaptability to Changes,Decisiveness
id,,,,,,,,,,,,,,,,,
5c1c54d7da0e492bb4580c7b,5.5,5.3,5.8,5.9,6.0,5.68,4.666667,4.000000,3.333333,5.333333,4.666667,3.000000,4.666667,3.000000,2.000000,4.000000,6.000000
5c1c55cfda0e492bb4580c7d,5.6,4.1,5.3,4.2,4.6,5.42,5.000000,4.500000,4.500000,4.000000,5.500000,3.000000,5.000000,1.500000,1.000000,5.000000,4.000000
5c2315fb58497715f43f3dd7,3.9,3.1,2.5,3.4,3.5,3.06,2.000000,3.500000,2.000000,1.500000,2.500000,3.000000,3.000000,1.000000,4.500000,2.000000,2.500000
5c2316ec58497715f43f3dd8,5.5,3.6,5.1,4.1,3.7,5.26,1.000000,3.500000,2.500000,1.500000,2.000000,2.500000,2.000000,1.000000,5.000000,2.500000,4.000000
5c380026ab15e621f457e791,5.0,4.2,4.3,4.6,4.8,4.58,3.666667,4.666667,3.000000,1.666667,4.000000,1.666667,2.000000,1.333333,3.666667,1.333333,2.000000
5c4a955bcd053b079445153a,5.1,5.2,4.8,4.8,5.5,4.92,4.000000,4.600000,4.200000,4.000000,5.400000,4.200000,5.250000,4.600000,1.000000,5.250000,4.000000
5c4aa60e9dd0fb08d0c4545c,4.0,3.8,3.9,3.0,3.9,3.94,3.800000,3.000000,3.000000,2.750000,2.750000,1.600000,2.750000,1.400000,5.500000,4.000000,3.000000
5c4aa6229dd0fb08d0c4545d,4.1,4.6,3.2,3.2,4.4,3.56,3.000000,4.000000,2.333333,3.000000,4.333333,2.000000,3.000000,1.666666,5.000000,1.333333,6.000000
5c4aa6349dd0fb08d0c4545e,3.8,3.5,3.8,3.8,3.9,3.80,2.200000,2.000000,2.300000,1.750000,2.900000,1.000000,1.500000,3.000000,4.500000,2.000000,3.800000


In [76]:
tot_table.to_excel("mitsubishi_missing.xlsx")

In [31]:
reports_table_ip = reports_table[['Empathy','Appropriate Behaviour Understanding','Conscientiousness','Emotional Understanding','Collegial Behavior','Teamwork']]
reports_table_ip.dropna(axis=0,how='any',inplace=True)
print(reports_table_ip.shape)
reports_table_ip.head()

(0, 6)


,Empathy,Appropriate Behaviour Understanding,Conscientiousness,Emotional Understanding,Collegial Behavior,Teamwork
id,,,,,,


In [32]:
reports_table_ip = reports_table_ip.dropna(axis=0).round(0)
print('before:',reports_table_ip.shape)
print('after:',reports_table_ip.drop_duplicates().shape)
#reports_table_ip.drop_duplicates()

before: (0, 6)
after: (0, 6)


In [33]:
reports_table_p = reports_table[['Decisiveness','Adaptability to Changes','Risk Taking','Performance under Stress','Conscientiousness']]
reports_table_p.dropna(axis=0,subset=['Decisiveness','Adaptability to Changes','Risk Taking','Performance under Stress'],how='any',inplace=True)
print(reports_table_p.shape)
reports_table_p.head(20)

(38, 5)


,Decisiveness,Adaptability to Changes,Risk Taking,Performance under Stress,Conscientiousness
id,,,,,
5c1c54d7da0e492bb4580c7b,6.000000,4.000000,2.000000,3.000000,NaN
5c1c55cfda0e492bb4580c7d,4.000000,5.000000,1.000000,1.500000,NaN
5c2315fb58497715f43f3dd7,2.500000,2.000000,4.500000,1.000000,NaN
5c2316ec58497715f43f3dd8,4.000000,2.500000,5.000000,1.000000,NaN
5c380026ab15e621f457e791,2.000000,1.333333,3.666667,1.333333,2.5
5c4a955bcd053b079445153a,4.000000,5.250000,1.000000,4.600000,NaN
5c4aa60e9dd0fb08d0c4545c,3.000000,4.000000,5.500000,1.400000,NaN
5c4aa6229dd0fb08d0c4545d,6.000000,1.333333,5.000000,1.666666,NaN
5c4aa6349dd0fb08d0c4545e,3.800000,2.000000,4.500000,3.000000,NaN


In [17]:
reports_table_p = reports_table_p.dropna(axis=0).round(0)
reports_table_p.replace({1.0:2.0,5.0:6.0},inplace=True)
print('before:',reports_table_p.shape)
print('after:',reports_table_p.drop_duplicates().shape)
reports_table_p.drop_duplicates()

before: (1, 5)
after: (1, 5)


,Decisiveness,Adaptability to Changes,Risk Taking,Performance under Stress,Conscientiousness
id,,,,,
5c380026ab15e621f457e791,2.0,2.0,4.0,2.0,2.0


In [18]:
reports_table_p.drop_duplicates().to_csv(path_dataset+os.sep+'unique_personality_reports_1=2_5=6.csv')

In [34]:
a = np.random.rand(80)
for i in a:
   # print(round((i-.5)*.6,2))

SyntaxError: unexpected EOF while parsing (<ipython-input-34-31f5cfb4e403>, line 3)

In [8]:
observation_table.columns

Index(['id', 'company_name', 'Time Management', 'Teamwork', 'Quantitative reasoning', 'Quantity vs Quality', 'Adaptability to changes', 'Agreeableness', 'Decisiveness', 'Purposefulness Vs. Hesitant', 'Adaptability to Changes', 'Risk Taking', 'Stress Tolerance', 'Attention: Coping with Distractions', 'Goal orientation', 'Coping with stress', 'Methodicalness', 'Performance IQ ', 'Thoroughness', 'Empathy', 'Processing Speed', 'Performance under stress', 'Processing speed', 'Perceptual Organization', 'Precision', 'Monitoring', 'Orderliness', 'Memory', 'Performance under Stress', 'Self Monitoring', 'Working memory', 'Self-efficacy', 'Spatial reasoning', 'Verbal reasoning', 'Appropriate Behaviour Understanding', 'Quantitative Reasoning', 'Conscientiousness', 'Cognitive ability total score', 'Planning', 'Verbal Reasoning', 'Performance IQ', 'Emotional Understanding', 'Collegial Behavior', 'Intelligence Quotient'], dtype='object')

In [10]:
observation_table.isnull().sum()/observation_table.shape[0]*100


id                                      0.000000
company_name                            0.000000
Time Management                         3.934010
Teamwork                               40.862944
Quantitative reasoning                 77.664975
Quantity vs Quality                    79.441624
Adaptability to changes                81.725888
Agreeableness                          97.715736
Decisiveness                           65.609137
Purposefulness Vs. Hesitant            37.817259
Adaptability to Changes                94.796954
Risk Taking                            77.664975
Stress Tolerance                       55.837563
Attention: Coping with Distractions    65.736041
Goal orientation                       65.609137
Coping with stress                     97.715736
Methodicalness                         65.609137
Performance IQ                         98.604061
Thoroughness                           65.609137
Empathy                                70.050761
Processing Speed    

In [53]:
#Export relevant id
#observation_table['id'].to_csv("id.csv", index = False)


In [76]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [108]:
import json
file = "W:\\Work\\Assense\\ML\\DataFrames\\ang_25_03_19.json"
with open(file) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('ang.s_sAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_ang = df_gnr.sum()
df_ang/15

(448, 438)


ang.s_sAccuracy.0     22.000000
ang.s_sAccuracy.1     22.600000
ang.s_sAccuracy.2     18.000000
ang.s_sAccuracy.3     26.500000
ang.s_sAccuracy.4     14.700000
ang.s_sAccuracy.5     21.800000
ang.s_sAccuracy.6     21.466667
ang.s_sAccuracy.7     21.800000
ang.s_sAccuracy.8     24.300000
ang.s_sAccuracy.9     14.700000
ang.s_sAccuracy.10    18.566667
ang.s_sAccuracy.11    10.100000
ang.s_sAccuracy.12    17.733333
ang.s_sAccuracy.13    19.766667
ang.s_sAccuracy.14     9.800000
ang.s_sAccuracy.15    20.400000
ang.s_sAccuracy.16    11.666667
dtype: float64

In [92]:
df_gnr['ang.s_sAccuracy']/15*5+1

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [107]:
file1 = "W:\\Work\\Assense\\ML\\DataFrames\\pzl_24_03_19.json"
with open(file1) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('pzl.r_sAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_pzl = df_gnr.sum()
df_pzl/24

(453, 1637)


pzl.r_sAccuracy.0      8.625000
pzl.r_sAccuracy.1     16.708333
pzl.r_sAccuracy.2     15.291667
pzl.r_sAccuracy.3     16.583333
pzl.r_sAccuracy.4     18.458333
pzl.r_sAccuracy.5     15.750000
pzl.r_sAccuracy.6     17.250000
pzl.r_sAccuracy.7     13.916667
pzl.r_sAccuracy.8     14.958333
pzl.r_sAccuracy.9     17.666667
pzl.r_sAccuracy.10    13.541667
pzl.r_sAccuracy.11    17.458333
pzl.r_sAccuracy.12    15.666667
pzl.r_sAccuracy.13    16.333333
pzl.r_sAccuracy.14    11.250000
pzl.r_sAccuracy.15    12.750000
pzl.r_sAccuracy.16    18.583333
pzl.r_sAccuracy.17    17.625000
pzl.r_sAccuracy.18    12.541667
pzl.r_sAccuracy.19    15.666667
pzl.r_sAccuracy.20    17.583333
pzl.r_sAccuracy.21    18.375000
pzl.r_sAccuracy.22    16.083333
pzl.r_sAccuracy.23    14.541667
dtype: float64

In [109]:
file2 = "W:\\Work\\Assense\\ML\\DataFrames\\shp_24_03_19.json"
with open(file2) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('shp.r_sAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_shp = df_gnr.sum()
df_shp/11

(432, 473)


shp.r_sAccuracy.0     33.636364
shp.r_sAccuracy.1     26.272727
shp.r_sAccuracy.2     22.272727
shp.r_sAccuracy.3     24.181818
shp.r_sAccuracy.4     27.363636
shp.r_sAccuracy.5     26.090909
shp.r_sAccuracy.6     29.181818
shp.r_sAccuracy.7     26.818182
shp.r_sAccuracy.8     14.636364
shp.r_sAccuracy.9      5.000000
shp.r_sAccuracy.10     6.545455
shp.r_sAccuracy.11     6.909091
dtype: float64

In [110]:
file3 = "W:\\Work\\Assense\\ML\\DataFrames\\pcs_24_03_19.json"
with open(file3) as train_file:
    dict_train = json.load(train_file)
sol_ac=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('pcs.r_fAccuracy\..', col):
        sol_ac.append(col)

sol_ac.sort(key=natural_keys)
df_gnr= df[sol_ac]
df_pcs = df_gnr.sum()
df_pcs/87

(166, 3640)


pcs.r_fAccuracy.0     1.908046
pcs.r_fAccuracy.1     1.908046
pcs.r_fAccuracy.2     1.823755
pcs.r_fAccuracy.3     1.858238
pcs.r_fAccuracy.4     1.888889
pcs.r_fAccuracy.5     1.885057
pcs.r_fAccuracy.6     1.885057
pcs.r_fAccuracy.7     1.888889
pcs.r_fAccuracy.8     1.896552
pcs.r_fAccuracy.9     1.865900
pcs.r_fAccuracy.10    1.896552
pcs.r_fAccuracy.11    1.896552
pcs.r_fAccuracy.12    1.865900
pcs.r_fAccuracy.13    1.877395
pcs.r_fAccuracy.14    1.640805
pcs.r_fAccuracy.15    1.830460
pcs.r_fAccuracy.16    1.744253
pcs.r_fAccuracy.17    1.813218
pcs.r_fAccuracy.18    1.747126
pcs.r_fAccuracy.19    1.660920
pcs.r_fAccuracy.20    1.473563
pcs.r_fAccuracy.21    1.756322
pcs.r_fAccuracy.22    1.671264
pcs.r_fAccuracy.23    1.781609
pcs.r_fAccuracy.24    1.742529
pcs.r_fAccuracy.25    1.668966
pcs.r_fAccuracy.26    1.566667
pcs.r_fAccuracy.27    1.662069
pcs.r_fAccuracy.28    1.701916
pcs.r_fAccuracy.29    1.675862
pcs.r_fAccuracy.30    1.692337
pcs.r_fAccuracy.31    1.748276
pcs.r_fA

In [118]:
#remove YARD on specific subjects (she entereher scores as YARD2 instead)
observation_table = observation_table[~np.logical_and(observation_table.Username=='YARD',
               list(map(lambda x: x in ['A28','A21','M6','M8'],observation_table.MissionId)))]


In [91]:
# pivot table
#scores = pd.pivot_table(observation_table,values='TotalScore',index=['Question','MissionId'],columns='Username',aggfunc=np.max)
#scores.head()